In [1]:
!pip install tiktoken

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu


In [3]:
import tiktoken

In [4]:
tokenizer = tiktoken.get_encoding("gpt2")

text = "Hello, how are you? <|endoftext|> It's a sunny day."
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

strings = tokenizer.decode(integers)
print(strings)

[15496, 11, 703, 389, 345, 30, 220, 50256, 632, 338, 257, 27737, 1110, 13]
Hello, how are you? <|endoftext|> It's a sunny day.


In [2]:
import os
import urllib.request

if not os.path.exists("data/the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "data/the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)
    
with open("data/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print(raw_text[:50])

I HAD always thought Jack Gisburn rather a cheap g


In [5]:
import torch
import ttnn

2025-04-19 14:29:54.135 | DEBUG    | ttnn:<module>:83 - Initial ttnn.CONFIG:
Config{cache_path=/home/avgdev/.cache/ttnn,model_cache_path=/home/avgdev/.cache/ttnn/models,tmp_dir=/tmp/ttnn,enable_model_cache=false,enable_fast_runtime_mode=true,throw_exception_on_fallback=false,enable_logging=false,enable_graph_report=false,enable_detailed_buffer_report=false,enable_detailed_tensor_report=false,enable_comparison_mode=false,comparison_mode_should_raise_exception=false,comparison_mode_pcc=0.9999,root_report_path=generated/ttnn/reports,report_name=std::nullopt,std::nullopt}


In [6]:
from scripts.prepare_data import create_dataloader_v1

dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)

batches = [ttnn.from_torch(batch, dtype=ttnn.uint32) for batch in first_batch]

print(batches)

[ttnn.Tensor([[   40,   367,  ...,  2885,  1464]], shape=Shape([1, 4]), dtype=DataType::UINT32, layout=Layout::ROW_MAJOR), ttnn.Tensor([[  367,  2885,  ...,  1464,  1807]], shape=Shape([1, 4]), dtype=DataType::UINT32, layout=Layout::ROW_MAJOR)]


In [7]:
def to_ttnn_batch(torch_batch, device=None):
    return ttnn.from_torch(torch_batch, dtype=ttnn.uint32, device=device, layout=ttnn.ROW_MAJOR_LAYOUT)

In [8]:
second_batch = next(data_iter)
print([to_ttnn_batch(batch) for batch in second_batch])

[ttnn.Tensor([[  367,  2885,  ...,  1464,  1807]], shape=Shape([1, 4]), dtype=DataType::UINT32, layout=Layout::ROW_MAJOR), ttnn.Tensor([[ 2885,  1464,  ...,  1807,  3619]], shape=Shape([1, 4]), dtype=DataType::UINT32, layout=Layout::ROW_MAJOR)]


In [9]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", to_ttnn_batch(inputs))
print("\nTargets:\n", to_ttnn_batch(targets))

Inputs:
 ttnn.Tensor([[   40,   367,  ...,  2885,  1464],
             [ 1807,  3619,  ...,   402,   271],
             ...,
             [ 1049,  5975,  ...,   284,   502],
             [  284,  3285,  ...,   326,    11]], shape=Shape([8, 4]), dtype=DataType::UINT32, layout=Layout::ROW_MAJOR)

Targets:
 ttnn.Tensor([[  367,  2885,  ...,  1464,  1807],
             [ 3619,   402,  ...,   271, 10899],
             ...,
             [ 5975,   284,  ...,   502,   284],
             [ 3285,   326,  ...,    11,   287]], shape=Shape([8, 4]), dtype=DataType::UINT32, layout=Layout::ROW_MAJOR)


In [10]:
input_ids = torch.tensor([2, 3, 5, 1])
vocab_size = 6
output_dim = 3
torch.manual_seed(123)

embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

# we need to recreate this with ttnn



In [11]:
device_id = 0
device = ttnn.open_device(device_id=device_id)

                 Device | INFO     | Opening user mode device driver
2025-04-19 14:29:55.369 | INFO     | SiliconDriver   - Opened PCI device 0; KMD version: 1.33.0, IOMMU: disabled

2025-04-19 14:29:55.382 | INFO     | SiliconDriver   - Opened PCI device 0; KMD version: 1.33.0, IOMMU: disabled
2025-04-19 14:29:55.383 | INFO     | SiliconDriver   - Harvesting mask for chip 0 is 0x200 (physical layout: 0x1, logical: 0x200, simulated harvesting mask: 0x0).
2025-04-19 14:29:55.384 | INFO     | SiliconDriver   - Opened PCI device 0; KMD version: 1.33.0, IOMMU: disabled
2025-04-19 14:29:55.385 | INFO     | SiliconDriver   - Detected PCI devices: [0]
2025-04-19 14:29:55.385 | INFO     | SiliconDriver   - Using local chip ids: {0} and remote chip ids {}
2025-04-19 14:29:55.412 | INFO     | SiliconDriver   - Software version 6.0.0, Ethernet FW version 6.14.0 (Device 0)
                  Metal | INFO     | Initializing device 0. Program cache is NOT enabled
                  Metal | INFO     | 

New chip! We now have 1 chips
Chip initialization complete (found )
Chip initializing complete...
 ARC

 [4/4] DRAM

 [16/16] ETH

 CPU

Chip detection complete (found )


In [12]:
# first we need to create an input tensor - this is just tokens. let's make some random stuff

input_tensor_ttnn = ttnn.from_torch(input_ids, dtype=ttnn.uint32, layout=ttnn.ROW_MAJOR_LAYOUT, device=device)

print(input_tensor_ttnn)

ttnn.Tensor([    2,     3,  ...,     5,     1], shape=Shape([4]), dtype=DataType::UINT32, layout=Layout::ROW_MAJOR)


In [13]:
# next we need an initialization of random weight matrices:
weight_torch = torch.randn(vocab_size, output_dim)
weight_torch

weight_ttnn = ttnn.from_torch(weight_torch, dtype=ttnn.bfloat16, device=device)
weight_ttnn

ttnn.Tensor([[-2.14062,  1.05469, -1.21875],
             [ 1.31250,  1.05469,  0.13867],
             ...,
             [ 0.18359,  0.22949,  0.61719],
             [-0.28711,  0.82031,  0.15137]], shape=Shape([6, 3]), dtype=DataType::BFLOAT16, layout=Layout::ROW_MAJOR)

In [14]:
embedding_ttnn = ttnn.embedding(input_tensor_ttnn, weight_ttnn)
embedding_ttnn

ttnn.Tensor([[-0.20410, -2.26562, -0.91406],
             [-0.41992, -0.66016, -0.79688],
             ...,
             [-0.28711,  0.82031,  0.15137],
             [ 1.31250,  1.05469,  0.13867]], shape=Shape([4, 3]), dtype=DataType::BFLOAT16, layout=Layout::ROW_MAJOR)

In [15]:
output = ttnn.from_device(embedding_ttnn)
output

ttnn.Tensor([[-0.20410, -2.26562, -0.91406],
             [-0.41992, -0.66016, -0.79688],
             ...,
             [-0.28711,  0.82031,  0.15137],
             [ 1.31250,  1.05469,  0.13867]], shape=Shape([4, 3]), dtype=DataType::BFLOAT16, layout=Layout::ROW_MAJOR)

In [16]:
ttnn.close_device(device)

                  Metal | INFO     | Closing device 0
                  Metal | INFO     | Disabling and clearing program cache on device 0


In [17]:
# note that output on CPu is equivalent to : embedding_layer(input_ids) in torch
output

ttnn.Tensor([[-0.20410, -2.26562, -0.91406],
             [-0.41992, -0.66016, -0.79688],
             ...,
             [-0.28711,  0.82031,  0.15137],
             [ 1.31250,  1.05469,  0.13867]], shape=Shape([4, 3]), dtype=DataType::BFLOAT16, layout=Layout::ROW_MAJOR)

In [18]:
def safe_view(ttnn_tensor, seq_len, out_dim):
    rm = ttnn.to_layout(ttnn_tensor, ttnn.ROW_MAJOR_LAYOUT)
    cpu = ttnn.to_torch(rm)
    return cpu[:, :seq_len, :out_dim]

In [19]:
vocab_size = 50257
output_dim = 256

# Open the device
device_id = 0
device = ttnn.open_device(device_id=device_id)

# Get the next batch of inputs and targets. This should be a shape of (8, 4)
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

# Next let's convert these to be ttnn tensor friendly and send to hardware
inputs_ttnn = ttnn.to_device(to_ttnn_batch(inputs, device=device), device)
targets_ttnn = ttnn.to_device(to_ttnn_batch(targets, device=device), device=device)

# We will now create the token embedding matrix.
# Create randomized weight matrix first 
weight_ttnn = ttnn.to_device(
    ttnn.from_torch(
        torch.randn(vocab_size, output_dim),
        dtype=ttnn.bfloat16,
        layout=ttnn.ROW_MAJOR_LAYOUT
    ),
    device
)

# create the token_embedding matrix with weight_ttnn and inputs_ttnn using ttnn.embedding
# the result should be (8, 4, 256)
token_embedding_ttnn = ttnn.embedding(inputs_ttnn, weight_ttnn)
print("TOKEN_EMBEDDING TTNN", token_embedding_ttnn)

# Next, create the positional embedding matrix. this should end up being (4, 256)
# First, we need to create the randomized weights
context_length = max_length
pos_weight_ttnn = ttnn.to_device(
    ttnn.from_torch(
        torch.randn(context_length, output_dim),
        dtype=ttnn.bfloat16,
        layout=ttnn.ROW_MAJOR_LAYOUT
    ),
    device
)

# Create the positional inputs matrix
pos_input_ttnn = ttnn.to_device(ttnn.arange(end=max_length, dtype=ttnn.uint32), device)

# finally create the positional embedding matrix
pos_embedding_ttnn = ttnn.embedding(pos_input_ttnn, pos_weight_ttnn)
print("POSITION TTNN", pos_embedding_ttnn)

# Combine the token embeddings and positional embeddings to create the input embeddings
pos_embedding_ttnn = ttnn.reshape(pos_embedding_ttnn, (1, 4, 256))
pos_embedding_ttnn = ttnn.repeat_interleave(pos_embedding_ttnn, repeats=8, dim=0)
input_embeddings_ttnn = ttnn.add(ttnn.tilize(token_embedding_ttnn), ttnn.tilize(pos_embedding_ttnn))

print(ttnn.untilize(input_embeddings_ttnn))

ttnn.close_device(device)

                  Metal | INFO     | Initializing device 0. Program cache is NOT enabled
                  Metal | INFO     | AI CLK for device 0 is:   1000 MHz
TOKEN_EMBEDDING TTNN ttnn.Tensor([[[ 0.42383,  0.27930,  ...,  0.43359, -1.03125],
              [-0.94141,  3.17188,  ...,  0.85938,  1.13281],
              ...,
              [-0.14160,  0.51953,  ..., -0.86719,  0.89062],
              [-0.72266,  0.56641,  ...,  1.31250,  0.19043]],

             [[-0.82031, -0.33594,  ..., -1.05469, -1.54688],
              [ 1.17969, -0.96875,  ..., -0.21387,  0.09814],
              ...,
              [-0.40430,  0.41797,  ..., -1.25000,  0.73047],
              [-0.43164, -0.83594,  ...,  1.38281, -0.69531]],

             ...,

             [[-0.87109,  0.05957,  ..., -0.64062, -0.57031],
              [-0.36523,  0.61328,  ..., -0.84375, -0.32031],
              ...,
              [-0.23340,  2.00000,  ..., -0.04639,  0.44141],
              [-0.85938, -0.04541,  ...,  0.37500, -0.79